In [1]:
import uuid
import random

from tqdm import tqdm
from threading import Thread
from threading import Thread, Lock

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
import time

In [2]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.TWO,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [3]:
res = get_all_reservations(session)
for r in res:
    reservation_id = r.reservation_id
    user_id = r.user_id
    user_name = r.user_name
    book_id = r.book_id
    book_name = r.book_name
    
reservation_id

UUID('8efc2616-0d5d-4d20-9b62-604b8c9db02f')

In [4]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user1 is not None and user2 is None and user1.user_id != user.user_id:
        user2 = user

    if user1 is None:
        user1 = user
    
(user1, user2)

(Row(user_id=UUID('37e94fcc-d6ce-4c9c-a586-3d2956632656'), reservation_id=UUID('027a896c-0c43-4866-a5aa-81b94bf6ebba'), book_id=UUID('5ff8a8fd-17b6-4323-b7c7-3e137fd6fef4'), book_name='Book 666', user_name='User 83'),
 Row(user_id=UUID('6acb9f75-a28b-4fae-9547-b0edfe27c8ac'), reservation_id=UUID('1695de29-65fc-4a62-9079-2db0dd927db2'), book_id=UUID('45ccdc81-d47e-4871-ab19-ba4334f80ced'), book_name='Book 95', user_name='User 2'))

# New Tests

In [2]:
def test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation(session, reservation_id, user_id, user_name, book_name, book_id)
    
    #Added for testing:
    time.sleep(0.5)
    reservation = get_reservation_by_id(session, reservation_id)
    assert reservation is not None

def test_same_request(session, reservation_id, new_book_id, repeat_num=10_000):
    # Runs an update of the book in a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        update_reservation(session, reservation_id, new_book_id)
    
    #Added for testing
    time.sleep(0.5)
    reservation = get_reservation_by_id(session, reservation_id)
    assert reservation.book_id == new_book_id

def test_update_reservations(session, num_reservations=1000):
    # Updates a 1000 reservations

    reservations = list(get_all_reservations(session))
    books = get_all_books(session)
    available_books = []
    for book in books:
        if not book.is_reserved:
            available_books.append(book)

    assert len(reservations) >= num_reservations, f"There should be at least a {num_reservations} existing reservations"
    assert len(available_books) >= num_reservations, f"There should be at least a {num_reservations} available books"

    for i in tqdm(range(num_reservations)):
        reservation = reservations[i]
        new_book = available_books[i]

        update_reservation(session, reservation.reservation_id, new_book.book_id)

def test_make_all_reservations_two_users(user1, user2):
    # Simulates two users reserving all possible books

    def make_all_reservations(user_id, user_name):
        # Simulates user making all possible reservations one by one

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        # Update available book state
        books = get_all_books(user_session)
        available_books = [book for book in books if not book.is_reserved]
    
        while len(available_books) > 0:
            selected_random_book = random.choice(available_books)
            
            try:
                add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
            except:
                pass
            
            books = get_all_books(user_session)
            available_books = [book for book in books if not book.is_reserved]

        user_session.shutdown()

    thread1 = Thread(target=make_all_reservations, args=[user1.user_id, user1.user_name])
    thread2 = Thread(target=make_all_reservations, args=[user2.user_id, user2.user_name])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

def test_make_random_requests_two_users(user1, user2, num_repeats=10_000):
    # Simulates two users making 10_000 total random requests

    def make_random_requests(user_id, user_name, num_actions):
        # Simulates a user making random requests num_actions number of times

        user_cluster = Cluster(CLUSTER_IDS)
        user_session = user_cluster.connect(KEYSPACE)

        possible_requests = ["update_reservation_book", "update_reservation_user", "cancel_reservation", "make_reservation"]

        while num_actions > 0:
            
            num_actions -= 1
            selected_action = random.choice(possible_requests)

            # Update DB state information
            reservations = list(get_all_reservations(user_session))

            # If no reservations left, make a reservation
            if len(reservations) == 0:
                selected_action = "make_reservation"

            books = list(get_all_books(user_session))
            users = list(get_all_users(user_session))
            
            if selected_action == "update_reservation_book":
                reservation = random.choice(reservations)
                book = random.choice(books)

                # Does not assume book is available
                update_reservation(user_session, reservation.reservation_id, book.book_id)

            elif selected_action == "update_reservation_user":
                reservation = random.choice(reservations)
                user = random.choice(users)

                # Does not assume user does not possess reservation already
                update_reservation_user(user_session, reservation.reservation_id, user.user_id, user.user_name)

            elif selected_action == "cancel_reservation":
                reservation = random.choice(reservations)
                cancel_reservation(user_session, reservation.reservation_id)

            elif selected_action == "make_reservation":
                available_books = [book for book in books if not book.is_reserved]

                # There are books available
                if len(available_books) > 0:
                    selected_random_book = random.choice(available_books)

                    try:
                        add_reservation(user_session, uuid.uuid4(), user_id, user_name, selected_random_book.book_name, selected_random_book.book_id)
                    except:
                        pass
                    

        user_session.shutdown()

    thread1 = Thread(target=make_random_requests, args=[user1.user_id, user1.user_name, num_repeats//2])
    thread2 = Thread(target=make_random_requests, args=[user2.user_id, user2.user_name, num_repeats//2])

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

In [6]:
test_reserve_cancel(session, reservation_id, user_id, user_name, book_id, book_name)

100%|██████████| 10000/10000 [01:01<00:00, 163.09it/s]


In [7]:
test_same_request(session, reservation_id, book_id)

100%|██████████| 10000/10000 [00:25<00:00, 397.54it/s]


In [8]:
test_update_reservations(session)

100%|██████████| 1000/1000 [00:10<00:00, 94.08it/s]


In [9]:
test_make_all_reservations_two_users(user1, user2)

In [11]:
session.shutdown()
cluster.shutdown()

In [3]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.TWO,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [4]:
res = get_all_reservations(session)
for r in res:
    reservation_id = r.reservation_id
    user_id = r.user_id
    user_name = r.user_name
    book_id = r.book_id
    book_name = r.book_name
    
reservation_id

UUID('25fe28c9-cb3c-400e-925a-28ce20fe7844')

In [5]:
users = get_all_users(session)
user1, user2 = None, None
for user in users:
    if user1 is not None and user2 is None and user1.user_id != user.user_id:
        user2 = user

    if user1 is None:
        user1 = user
    
(user1, user2)

(Row(user_id=UUID('b12bedab-69d8-4225-a749-87aea221292b'), reservation_id=UUID('008d44a8-8407-4981-810a-1b01e84c7b84'), book_id=UUID('084c9b45-a2f9-4280-80ab-dd7f3551786a'), book_name='Book 1352', user_name='User 20'),
 Row(user_id=UUID('f237a743-f796-425c-90df-e19f71448be2'), reservation_id=UUID('006430a8-f67e-4b83-9c67-6e469398bb4e'), book_id=UUID('14c2bbcd-58ba-4f71-8127-be1926e805a6'), book_name='Book 939', user_name='User 64'))

In [6]:
# IMPORTANT: Resetting the database before this test and reconnecting (RUN delete_static_DB.py and then RUN create_tests_DB.py )
test_make_random_requests_two_users(user1, user2)

Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating a reservation: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error occurred while updating book reservation status: Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"
Error canceling the reservation:  Error from server: code=2200 [Invalid query] message="Invalid null value in condition for column book_id"


In [ ]:
session.shutdown()
cluster.shutdown()

# Prev Tests

In [ ]:
def add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id, timeout = 120):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:
            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            #append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)

In [ ]:
def add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id, timeout = 120):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:
            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)

In [ ]:
class Add_Reservation_Handler:
    def __init__(self, session):
        self.insert_data = []  # Initialized as an empty list
        self.counter = 0  # Initialized as 0
        self.session = session

    def add_data(self, data):
        self.insert_data.append(data)
        self.counter += 1

    def display_data(self):
        print("Inserted Data:", self.insert_data)
        print("Counter:", self.counter)

    def batch_set_book_reserved(self, session, book_id, reserved = True, timeout = 120):
        pass

    def add_batch_reservation(self, reservation_id, user_id, user_name, book_name, book_id, batch_size = 1000, forceful_exec = False, timeout = 120):
        try:
            book = get_book(self.session, book_id = book_id)
            if book is not None and not book.is_reserved:
                self.insert_data.append((reservation_id, user_id, user_name, book_name, book_id))
                self.counter += 1
                if forceful_exec or self.counter == batch_size:
                    self.batch_set_book_reserved(book_id=book_id, reserved = True, timeout = timeout)
                    self.batch_append_user_reservation(self.session, user_id, user_name, reservation_id, timeout = timeout)
                    insert_reservation_query = self.session.prepare("""
                                                                    INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
                                                                    VALUES (?, ?, ?, ?, ?);""")
                    batch_add = BatchStatement()
                    for example in self.insert_data:
                        batch_add.add(insert_reservation_query, example[0], example[1], example[2], example[3], example[4])
                    
                    self.session.execute(batch_add, timeout=timeout)
                    self.counter = 0
                    self.insert_data = []
        except InvalidRequest as e:
            print("Error occurred while inserting reservation:", e)

In [ ]:
    print("Adding books")
    book_ids = []
    book_names = []
    insert_statement_books = session.prepare("""
        INSERT INTO books (book_id, book_name, is_reserved) VALUES (?, ?, ?)
    """)
    for i in range(200):
        batch_books = BatchStatement()
        for j in range(10):
            book_id = uuid.uuid4()
            book_ids.append(book_id)
            book_name = f'Book {i*10+j+1}'
            book_names.append(book_name)
            batch_books.add(insert_statement_books, (book_id, book_name, False))
        try:
            session.execute(batch_books, timeout=120)
        except InvalidRequest as e:
            raise e

In [ ]:
def add_batch_reservation(session, reservation_id, user_id, user_name, book_name, book_id, batch_size = 1000, forceful_exec = False, counter = 0):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:

            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)    
    return counter

def cancel_batch_reservation(session, reservation_id, batch_size = 1000, forceful_exec = False, prepared = False):
    pass

In [ ]:
def test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id)

def test_reserve_cancel_slow(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id)

In [ ]:
start_time = time.time()
test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [00:58<00:00, 169.56it/s]

Execution Time: 58.98241949081421


In [ ]:
start_time = time.time()
test_reserve_cancel_slow(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [03:18<00:00, 50.33it/s]

Execution Time: 198.7042052745819


In [ ]:
start_time = time.time()
test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [02:56<00:00, 56.51it/s]

Execution Time: 176.95874738693237
